# 📞 IVR Call Pattern Analysis with Local LLM

This notebook implements a complete system to:
1. Load and process self-service phone call data
2. Generate local embeddings using Ollama
3. Store in ChromaDB (local vector database)
4. Detect failure patterns and explain them with LLM

**Requirements:**
- Ollama installed and running locally
- Models downloaded (e.g.: `ollama pull llama3.1` and `ollama pull nomic-embed-text`)

## 1. Install Dependencies

In [ ]:
# Install required dependencies
!pip install chromadb pandas ollama scikit-learn matplotlib seaborn --quiet

In [ ]:
import pandas as pd
import numpy as np
import chromadb
import ollama
import json
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")

## 2. Ollama and ChromaDB Configuration

In [ ]:
@dataclass
class Config:
    """Centralized system configuration"""
    # Ollama Models
    EMBEDDING_MODEL: str = "nomic-embed-text"  # Model for embeddings
    LLM_MODEL: str = "llama3.1"                # Model for analysis/explanations
    
    # ChromaDB
    CHROMA_PERSIST_DIR: str = "./chroma_ivr_db"
    COLLECTION_NAME: str = "ivr_call_patterns"
    
    # Analysis
    TOP_K_SIMILAR: int = 5  # Number of similar cases to retrieve
    
config = Config()
print(f"📋 Configuration:")
print(f"   - Embedding model: {config.EMBEDDING_MODEL}")
print(f"   - LLM model: {config.LLM_MODEL}")
print(f"   - ChromaDB directory: {config.CHROMA_PERSIST_DIR}")

In [ ]:
# Verify Ollama connection
def verify_ollama_connection():
    """Verify that Ollama is running and check available models"""
    try:
        models = ollama.list()
        model_names = [m['name'].split(':')[0] for m in models.get('models', [])]
        print("✅ Ollama connected successfully")
        print(f"   Available models: {model_names}")
        
        # Check required models
        required = [config.EMBEDDING_MODEL, config.LLM_MODEL]
        for model in required:
            if model not in model_names:
                print(f"⚠️  Model '{model}' not found. Run: ollama pull {model}")
        
        return True
    except Exception as e:
        print(f"❌ Error connecting to Ollama: {e}")
        print("   Make sure Ollama is running: ollama serve")
        return False

verify_ollama_connection()

## 3. Sample Dataset Generation

If you already have your CSV, skip this cell and load your file directly.

In [ ]:
def generate_sample_dataset(n_calls: int = 500) -> pd.DataFrame:
    """
    Generate a sample dataset simulating IVR calls.
    Replace this with your actual CSV.
    """
    np.random.seed(42)
    
    # Define IVR flow steps
    ivr_steps = [
        "welcome_message",
        "language_selection",
        "authentication",
        "main_menu",
        "inquiry_submenu",
        "data_entry",
        "data_validation",
        "processing",
        "confirmation",
        "goodbye"
    ]
    
    # Possible errors by step
    error_types = {
        "authentication": ["auth_timeout", "invalid_credentials", "account_locked", "biometric_fail"],
        "data_entry": ["dtmf_timeout", "invalid_input", "speech_not_recognized", "too_many_retries"],
        "data_validation": ["data_mismatch", "expired_data", "system_unavailable"],
        "processing": ["backend_timeout", "service_unavailable", "transaction_failed"],
        "main_menu": ["no_input", "invalid_option", "confusion_detected"]
    }
    
    calls = []
    
    for i in range(n_calls):
        call_id = f"CALL_{i:05d}"
        
        # Determine if call will be successful (60% success rate)
        is_success = np.random.random() < 0.6
        
        if is_success:
            # Successful call - completes all steps
            steps_completed = ivr_steps.copy()
            step_results = {step: "success" for step in steps_completed}
            final_result = "success"
            failure_step = None
            failure_reason = None
            end_action = "completed"
        else:
            # Failed call - fails at some step
            fail_step_idx = np.random.choice([2, 3, 4, 5, 6, 7], p=[0.25, 0.1, 0.1, 0.25, 0.15, 0.15])
            steps_completed = ivr_steps[:fail_step_idx + 1]
            
            step_results = {step: "success" for step in steps_completed[:-1]}
            failure_step = steps_completed[-1]
            
            # Assign error type
            if failure_step in error_types:
                failure_reason = np.random.choice(error_types[failure_step])
            else:
                failure_reason = "unknown_error"
            
            step_results[failure_step] = "error"
            final_result = "error"
            
            # Final action
            end_action = np.random.choice(["hangup", "transfer_agent"], p=[0.4, 0.6])
        
        # Additional metadata
        call_duration = np.random.randint(30, 300) if is_success else np.random.randint(15, 180)
        retries = 0 if is_success else np.random.randint(1, 4)
        
        calls.append({
            "call_id": call_id,
            "timestamp": pd.Timestamp.now() - pd.Timedelta(days=np.random.randint(0, 30)),
            "steps_completed": json.dumps(steps_completed),
            "step_results": json.dumps(step_results),
            "final_result": final_result,
            "failure_step": failure_step,
            "failure_reason": failure_reason,
            "end_action": end_action,
            "call_duration_seconds": call_duration,
            "retry_count": retries,
            "customer_segment": np.random.choice(["premium", "standard", "basic"]),
            "call_type": np.random.choice(["billing", "support", "sales", "account_info"]),
            "hour_of_day": np.random.randint(8, 22),
            "day_of_week": np.random.choice(["monday", "tuesday", "wednesday", "thursday", "friday"])
        })
    
    return pd.DataFrame(calls)

# Generate sample dataset
df = generate_sample_dataset(500)
print(f"📊 Dataset generated: {len(df)} calls")
print(f"   - Successful: {len(df[df['final_result'] == 'success'])}")
print(f"   - Failed: {len(df[df['final_result'] == 'error'])}")
df.head()

In [ ]:
# ⬇️ ALTERNATIVE: Load your own CSV
# Uncomment and adjust according to your data structure

# df = pd.read_csv("your_file.csv")
# print(f"Dataset loaded: {len(df)} records")
# print(f"Columns: {df.columns.tolist()}")

## 4. Preprocessing and Text Representation Creation

In [ ]:
class CallDataProcessor:
    """
    Processes call data and generates text representations
    to create meaningful embeddings.
    """
    
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.processed_calls = []
    
    def create_call_narrative(self, row: pd.Series) -> str:
        """
        Converts a DataFrame row into a textual narrative
        that captures the complete call context.
        """
        # Parse JSON if necessary
        if isinstance(row['steps_completed'], str):
            steps = json.loads(row['steps_completed'])
        else:
            steps = row['steps_completed']
            
        if isinstance(row['step_results'], str):
            results = json.loads(row['step_results'])
        else:
            results = row['step_results']
        
        # Build narrative
        narrative_parts = [
            f"Call type: {row['call_type']}",
            f"Customer segment: {row['customer_segment']}",
            f"Day: {row['day_of_week']}, Hour: {row['hour_of_day']}:00",
            f"Duration: {row['call_duration_seconds']} seconds",
            "\nStep flow:"
        ]
        
        # Describe each step
        for i, step in enumerate(steps, 1):
            result = results.get(step, 'unknown')
            status = "✓" if result == "success" else "✗"
            narrative_parts.append(f"  {i}. {step}: {status} ({result})")
        
        # Final result
        narrative_parts.append(f"\nFinal result: {row['final_result'].upper()}")
        
        if row['final_result'] == 'error':
            narrative_parts.extend([
                f"Failure step: {row['failure_step']}",
                f"Failure reason: {row['failure_reason']}",
                f"End action: {row['end_action']}",
                f"Retry attempts: {row['retry_count']}"
            ])
        
        return "\n".join(narrative_parts)
    
    def create_pattern_signature(self, row: pd.Series) -> str:
        """
        Creates a more concise pattern signature for quick searching.
        """
        if isinstance(row['steps_completed'], str):
            steps = json.loads(row['steps_completed'])
        else:
            steps = row['steps_completed']
        
        signature_parts = [
            f"result:{row['final_result']}",
            f"steps:{len(steps)}",
            f"type:{row['call_type']}",
            f"segment:{row['customer_segment']}"
        ]
        
        if row['final_result'] == 'error':
            signature_parts.extend([
                f"fail_step:{row['failure_step']}",
                f"fail_reason:{row['failure_reason']}",
                f"end_action:{row['end_action']}"
            ])
        
        return " | ".join(signature_parts)
    
    def process_all(self) -> List[Dict]:
        """
        Processes all calls and returns list of documents.
        """
        self.processed_calls = []
        
        for idx, row in self.df.iterrows():
            doc = {
                "id": row['call_id'],
                "narrative": self.create_call_narrative(row),
                "signature": self.create_pattern_signature(row),
                "metadata": {
                    "call_id": row['call_id'],
                    "final_result": row['final_result'],
                    "failure_step": row['failure_step'] if pd.notna(row['failure_step']) else None,
                    "failure_reason": row['failure_reason'] if pd.notna(row['failure_reason']) else None,
                    "call_type": row['call_type'],
                    "customer_segment": row['customer_segment'],
                    "end_action": row['end_action']
                }
            }
            self.processed_calls.append(doc)
        
        return self.processed_calls

# Process data
processor = CallDataProcessor(df)
processed_calls = processor.process_all()

print(f"✅ {len(processed_calls)} calls processed")
print("\n📝 Example of generated narrative:")
print("-" * 50)
# Show example of failed call
failed_example = next(c for c in processed_calls if c['metadata']['final_result'] == 'error')
print(failed_example['narrative'])

## 5. Embedding Generation with Ollama

In [ ]:
class OllamaEmbeddings:
    """
    Generates embeddings using Ollama locally.
    """
    
    def __init__(self, model: str = "nomic-embed-text"):
        self.model = model
    
    def embed_single(self, text: str) -> List[float]:
        """Generate embedding for a single text."""
        response = ollama.embeddings(
            model=self.model,
            prompt=text
        )
        return response['embedding']
    
    def embed_batch(self, texts: List[str], show_progress: bool = True) -> List[List[float]]:
        """Generate embeddings for multiple texts."""
        embeddings = []
        total = len(texts)
        
        for i, text in enumerate(texts):
            emb = self.embed_single(text)
            embeddings.append(emb)
            
            if show_progress and (i + 1) % 50 == 0:
                print(f"   Processed: {i + 1}/{total}")
        
        return embeddings

# Initialize embedding generator
embedder = OllamaEmbeddings(model=config.EMBEDDING_MODEL)

# Quick test
test_emb = embedder.embed_single("embedding test")
print(f"✅ Embeddings working - Dimension: {len(test_emb)}")

## 6. ChromaDB Storage

In [ ]:
class IVRVectorStore:
    """
    Manages vector storage for IVR calls using ChromaDB.
    """
    
    def __init__(self, persist_dir: str, collection_name: str, embedder: OllamaEmbeddings):
        self.embedder = embedder
        
        # Initialize ChromaDB with persistence
        self.client = chromadb.PersistentClient(path=persist_dir)
        
        # Create or retrieve collection
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            metadata={"description": "IVR call patterns for failure analysis"}
        )
        
        print(f"✅ ChromaDB initialized")
        print(f"   Collection: {collection_name}")
        print(f"   Existing documents: {self.collection.count()}")
    
    def add_calls(self, processed_calls: List[Dict], batch_size: int = 100):
        """
        Add processed calls to the vector database.
        """
        print(f"\n📥 Indexing {len(processed_calls)} calls...")
        
        # Prepare data for ChromaDB
        ids = []
        documents = []
        metadatas = []
        
        for call in processed_calls:
            ids.append(call['id'])
            # Combine narrative and signature for the document
            documents.append(f"{call['narrative']}\n\nPattern: {call['signature']}")
            metadatas.append(call['metadata'])
        
        # Generate embeddings
        print("   Generating embeddings...")
        embeddings = self.embedder.embed_batch(documents)
        
        # Insert in batches
        for i in range(0, len(ids), batch_size):
            end_idx = min(i + batch_size, len(ids))
            
            self.collection.add(
                ids=ids[i:end_idx],
                documents=documents[i:end_idx],
                embeddings=embeddings[i:end_idx],
                metadatas=metadatas[i:end_idx]
            )
        
        print(f"✅ {len(ids)} calls indexed successfully")
        print(f"   Total in collection: {self.collection.count()}")
    
    def search_similar(self, query_text: str, n_results: int = 5, 
                       filter_result: Optional[str] = None) -> Dict:
        """
        Search for calls similar to a query.
        """
        # Generate query embedding
        query_embedding = self.embedder.embed_single(query_text)
        
        # Build optional filter
        where_filter = None
        if filter_result:
            where_filter = {"final_result": filter_result}
        
        # Search
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results,
            where=where_filter,
            include=["documents", "metadatas", "distances"]
        )
        
        return results
    
    def get_failure_patterns(self) -> Dict:
        """
        Get failure pattern statistics.
        """
        # Get all error documents
        results = self.collection.get(
            where={"final_result": "error"},
            include=["metadatas"]
        )
        
        # Analyze patterns
        failure_steps = Counter()
        failure_reasons = Counter()
        end_actions = Counter()
        
        for meta in results['metadatas']:
            if meta.get('failure_step'):
                failure_steps[meta['failure_step']] += 1
            if meta.get('failure_reason'):
                failure_reasons[meta['failure_reason']] += 1
            if meta.get('end_action'):
                end_actions[meta['end_action']] += 1
        
        return {
            "total_failures": len(results['metadatas']),
            "by_step": dict(failure_steps.most_common()),
            "by_reason": dict(failure_reasons.most_common()),
            "by_end_action": dict(end_actions.most_common())
        }
    
    def clear_collection(self):
        """Clear the collection (useful for re-training)."""
        self.client.delete_collection(self.collection.name)
        self.collection = self.client.create_collection(
            name=self.collection.name,
            metadata={"description": "IVR call patterns for failure analysis"}
        )
        print("🗑️ Collection cleared")

In [ ]:
# Initialize vector store
vector_store = IVRVectorStore(
    persist_dir=config.CHROMA_PERSIST_DIR,
    collection_name=config.COLLECTION_NAME,
    embedder=embedder
)

# Clear if we want to start fresh (optional)
# vector_store.clear_collection()

# Index calls only if collection is empty
if vector_store.collection.count() == 0:
    vector_store.add_calls(processed_calls)
else:
    print(f"ℹ️ Already {vector_store.collection.count()} documents indexed")

## 7. LLM Analysis Engine

In [ ]:
class IVRPatternAnalyzer:
    """
    Analyzes failure patterns using RAG + local LLM.
    """
    
    def __init__(self, vector_store: IVRVectorStore, llm_model: str):
        self.vector_store = vector_store
        self.llm_model = llm_model
    
    def _build_analysis_prompt(self, new_call: Dict, similar_cases: Dict) -> str:
        """
        Build the analysis prompt with similar cases context.
        """
        # Format similar cases
        similar_context = ""
        if similar_cases['documents'] and similar_cases['documents'][0]:
            for i, (doc, meta, dist) in enumerate(zip(
                similar_cases['documents'][0],
                similar_cases['metadatas'][0],
                similar_cases['distances'][0]
            ), 1):
                similar_context += f"\n--- Similar Case #{i} (similarity: {1-dist:.2f}) ---\n"
                similar_context += f"Result: {meta.get('final_result', 'N/A')}\n"
                if meta.get('failure_step'):
                    similar_context += f"Failure step: {meta['failure_step']}\n"
                    similar_context += f"Reason: {meta.get('failure_reason', 'N/A')}\n"
                similar_context += "\n"
        
        prompt = f"""You are an expert in IVR (Interactive Voice Response) system analysis and failure patterns in self-service phone calls.

Your task is to analyze a new call and explain the probable cause of failure based on:
1. The current call data
2. Patterns found in similar historical cases

## NEW CALL TO ANALYZE:
{new_call['narrative']}

## SIMILAR HISTORICAL CASES:
{similar_context if similar_context else 'No similar cases found.'}

## INSTRUCTIONS:
Provide a structured analysis that includes:

1. **DIAGNOSIS**: Explain what happened in this call and why it failed.

2. **IDENTIFIED PATTERN**: Describe if this failure follows a common pattern based on similar cases.

3. **PROBABLE ROOT CAUSE**: Identify the most likely cause of failure.

4. **RECOMMENDATIONS**: Suggest specific improvements to prevent this type of failure.

5. **CONFIDENCE LEVEL**: Indicate how confident you are in the diagnosis (High/Medium/Low) and why.

Be specific with step names and errors."""
        
        return prompt
    
    def analyze_call(self, call_data: Dict, find_similar: bool = True) -> str:
        """
        Analyze a call and generate failure explanation.
        """
        # Process the new call
        processor = CallDataProcessor(pd.DataFrame([call_data]))
        processed = processor.process_all()[0]
        
        # Search for similar cases
        similar_cases = {'documents': [[]], 'metadatas': [[]], 'distances': [[]]}
        if find_similar:
            similar_cases = self.vector_store.search_similar(
                processed['narrative'],
                n_results=config.TOP_K_SIMILAR,
                filter_result="error"  # Only search among failures
            )
        
        # Build prompt
        prompt = self._build_analysis_prompt(processed, similar_cases)
        
        # Generate analysis with LLM
        response = ollama.generate(
            model=self.llm_model,
            prompt=prompt,
            options={
                "temperature": 0.3,  # More deterministic for analysis
                "num_predict": 1000
            }
        )
        
        return response['response']
    
    def get_pattern_summary(self) -> str:
        """
        Generate a failure pattern summary using the LLM.
        """
        patterns = self.vector_store.get_failure_patterns()
        
        prompt = f"""Analyze the following IVR system failure statistics and provide an executive summary:

## FAILURE STATISTICS:
- Total failed calls: {patterns['total_failures']}

### Failures by step:
{json.dumps(patterns['by_step'], indent=2)}

### Failures by reason:
{json.dumps(patterns['by_reason'], indent=2)}

### User's final action:
{json.dumps(patterns['by_end_action'], indent=2)}

Provide:
1. The 3 most critical issues
2. Concerning patterns
3. Prioritized improvement recommendations

Respond concisely and actionably."""
        
        response = ollama.generate(
            model=self.llm_model,
            prompt=prompt,
            options={"temperature": 0.3}
        )
        
        return response['response']

# Initialize analyzer
analyzer = IVRPatternAnalyzer(vector_store, config.LLM_MODEL)
print("✅ Pattern analyzer initialized")

## 8. System Usage - Practical Examples

In [ ]:
# View pattern statistics
patterns = vector_store.get_failure_patterns()
print("📊 FAILURE PATTERN STATISTICS")
print("=" * 50)
print(f"\nTotal failed calls: {patterns['total_failures']}")
print(f"\nFailures by step:")
for step, count in patterns['by_step'].items():
    print(f"   {step}: {count}")
print(f"\nFailures by reason:")
for reason, count in patterns['by_reason'].items():
    print(f"   {reason}: {count}")

In [ ]:
# Generate executive summary with LLM
print("📋 EXECUTIVE PATTERN SUMMARY")
print("=" * 50)
summary = analyzer.get_pattern_summary()
print(summary)

### 8.1 Analyze a New Call

In [ ]:
# Simulate a new failed call to analyze
new_call = {
    "call_id": "NEW_001",
    "timestamp": pd.Timestamp.now(),
    "steps_completed": json.dumps([
        "welcome_message",
        "language_selection",
        "authentication"
    ]),
    "step_results": json.dumps({
        "welcome_message": "success",
        "language_selection": "success",
        "authentication": "error"
    }),
    "final_result": "error",
    "failure_step": "authentication",
    "failure_reason": "auth_timeout",
    "end_action": "transfer_agent",
    "call_duration_seconds": 45,
    "retry_count": 2,
    "customer_segment": "premium",
    "call_type": "billing",
    "hour_of_day": 14,
    "day_of_week": "wednesday"
}

print("🔍 ANALYZING NEW CALL")
print("=" * 50)
print(f"ID: {new_call['call_id']}")
print(f"Type: {new_call['call_type']}")
print(f"Failed at: {new_call['failure_step']}")
print(f"Reason: {new_call['failure_reason']}")
print("\n" + "=" * 50)
print("LLM ANALYSIS:")
print("=" * 50 + "\n")

analysis = analyzer.analyze_call(new_call)
print(analysis)

### 8.2 Search for Similar Cases

In [ ]:
# Search for cases similar to a specific pattern
query = "billing call that fails at authentication with timeout"

print(f"🔎 Searching for cases similar to: '{query}'")
print("=" * 50)

results = vector_store.search_similar(
    query_text=query,
    n_results=3,
    filter_result="error"
)

for i, (doc, meta, dist) in enumerate(zip(
    results['documents'][0],
    results['metadatas'][0],
    results['distances'][0]
), 1):
    print(f"\n--- Result #{i} (distance: {dist:.4f}) ---")
    print(f"Call ID: {meta['call_id']}")
    print(f"Failure step: {meta.get('failure_step', 'N/A')}")
    print(f"Reason: {meta.get('failure_reason', 'N/A')}")
    print(f"Type: {meta.get('call_type', 'N/A')}")

## 9. Quick Analysis Helper Function

In [ ]:
def quick_failure_analysis(
    call_type: str,
    failure_step: str,
    failure_reason: str,
    customer_segment: str = "standard",
    end_action: str = "hangup",
    retries: int = 1
) -> str:
    """
    Helper function for quick analysis of a failure case.
    
    Example:
        result = quick_failure_analysis(
            call_type="billing",
            failure_step="authentication",
            failure_reason="biometric_fail",
            customer_segment="premium"
        )
    """
    # Build steps up to failure
    all_steps = [
        "welcome_message",
        "language_selection",
        "authentication",
        "main_menu",
        "inquiry_submenu",
        "data_entry",
        "data_validation",
        "processing"
    ]
    
    # Find failure step index
    if failure_step in all_steps:
        fail_idx = all_steps.index(failure_step)
        steps_completed = all_steps[:fail_idx + 1]
    else:
        steps_completed = ["welcome_message", failure_step]
    
    # Build results
    step_results = {step: "success" for step in steps_completed[:-1]}
    step_results[failure_step] = "error"
    
    call = {
        "call_id": "QUICK_ANALYSIS",
        "timestamp": pd.Timestamp.now(),
        "steps_completed": json.dumps(steps_completed),
        "step_results": json.dumps(step_results),
        "final_result": "error",
        "failure_step": failure_step,
        "failure_reason": failure_reason,
        "end_action": end_action,
        "call_duration_seconds": 60,
        "retry_count": retries,
        "customer_segment": customer_segment,
        "call_type": call_type,
        "hour_of_day": 12,
        "day_of_week": "wednesday"
    }
    
    return analyzer.analyze_call(call)

print("✅ quick_failure_analysis() function available")

In [ ]:
# Example usage of the quick function
print("🚀 QUICK ANALYSIS")
print("=" * 50)

result = quick_failure_analysis(
    call_type="support",
    failure_step="data_entry",
    failure_reason="speech_not_recognized",
    customer_segment="basic",
    end_action="transfer_agent",
    retries=3
)

print(result)

## 10. Export and Save Results

In [ ]:
# Save the sample dataset for reference
df.to_csv("ivr_calls_sample.csv", index=False)
print("✅ Dataset saved to 'ivr_calls_sample.csv'")

# Save identified patterns
patterns = vector_store.get_failure_patterns()
with open("failure_patterns.json", "w") as f:
    json.dump(patterns, f, indent=2)
print("✅ Patterns saved to 'failure_patterns.json'")

---

## 📚 Usage Summary

### Main Components:

1. **`CallDataProcessor`**: Converts CSV data into textual narratives
2. **`OllamaEmbeddings`**: Generates embeddings using local model
3. **`IVRVectorStore`**: Stores and searches in ChromaDB
4. **`IVRPatternAnalyzer`**: Analyzes patterns with LLM

### Typical Flow:

```python
# 1. Load data
df = pd.read_csv("your_file.csv")

# 2. Process and index
processor = CallDataProcessor(df)
processed = processor.process_all()
vector_store.add_calls(processed)

# 3. Analyze new call
result = analyzer.analyze_call(new_call)

# 4. Or use quick function
result = quick_failure_analysis(
    call_type="billing",
    failure_step="authentication",
    failure_reason="timeout"
)
```

### Recommended Ollama Models:

- **Embeddings**: `nomic-embed-text` (fast and effective)
- **LLM**: `llama3.1` or `mistral` (good for analysis)
- **Lightweight alternative**: `phi3` or `gemma2` if you have limited resources